### Spark Sentence Detector

In [1]:
from pyspark.ml import PipelineModel
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from pyspark.ml import Pipeline
import pandas as pd
import re
pd.set_option("display.max_colwidth", 0)

In [2]:
from pyspark.sql import SparkSession

In [3]:
builder = SparkSession.builder\
        .appName("Spark NLP Licensed")\
        .master("local[*]")\
        .config("spark.driver.memory", "24G")\
        .config("spark.driver.maxResultSize", "2048GB")\
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")\
        .config("spark.kryoserializer.buffer.max", "2000M")\
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.3.2")

spark = builder.getOrCreate()

21/11/23 16:19:21 WARN Utils: Your hostname, kurubal resolves to a loopback address: 127.0.1.1; using 192.168.1.37 instead (on interface enp6s0)
21/11/23 16:19:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/kurubal/.ivy2/cache
The jars for the packages stored in: /home/kurubal/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4c7e3939-adfc-4811-b4b7-7641ed071fe7;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/kurubal/anaconda3/envs/pyspark_env/lib/python3.9/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.johnsnowlabs.nlp#spark-nlp_2.12;3.3.2 in central
	found com.typesafe#config;1.4.1 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found org.slf4j#slf4j-api;1.7.21 in central
	found com.navigamez#greex;1.0 in central
	found dk.brics.automaton#automaton;1.11-8 in central
	found org.json4s#json4s-ext_2.12;3.5.3 in central
	found joda-time#joda-time;2.9.5 in central
	found org.joda#joda-convert;1.8.1 in 

In [4]:
spark

In [5]:
#textFiles = spark.sparkContext.wholeTextFiles("oncology_notes/*")

#df = textFiles.toDF(schema=['path','text'])
#df.show()

In [6]:
df = spark.read.text('xaa.txt').toDF('text') # xaa.txt text data

In [7]:
df.select("text").head(1)

[Row(text='imle konuştular.')]

In [8]:
df.rdd.getNumPartitions()

8

In [9]:
df = df.repartition(200)

In [10]:
df.rdd.getNumPartitions()

200

In [11]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
sentencerDL = SentenceDetectorDLModel\
    .pretrained("sentence_detector_dl", "xx") \
    .setInputCols(["document"]) \
    .setOutputCol("sentences")

model = PipelineModel(stages=[documenter, sentencerDL])

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[ | ]sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[ / ]Download done! Loading the resource.
[ — ]

2021-11-23 16:19:56.971072: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-23 16:19:57.011288: I external/org_tensorflow/tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3599500000 Hz


[OK!]


In [12]:
result = model.transform(df)

In [13]:
result.show()

+--------------------+--------------------+--------------------+
|                text|            document|           sentences|
+--------------------+--------------------+--------------------+
|          Neredesin?|[{document, 0, 9,...|[{document, 0, 9,...|
|         Hapı yuttu!|[{document, 0, 10...|[{document, 0, 10...|
|    If Bu mekik, var|[{document, 0, 15...|[{document, 0, 16...|
| - Bembeyaz olmuştu.|[{document, 0, 18...|[{document, 0, 18...|
|Onu bozguna uğrat...|[{document, 0, 23...|[{document, 0, 23...|
|    Biraz yürüyelim.|[{document, 0, 15...|[{document, 0, 15...|
|     Olamaz, olamaz.|[{document, 0, 14...|[{document, 0, 14...|
|Bana ve onunla ol...|[{document, 0, 13...|[{document, 0, 13...|
| Nasıl başaracaksın?|[{document, 0, 18...|[{document, 0, 18...|
|Korkutucu değil a...|[{document, 0, 42...|[{document, 0, 42...|
|Şükürler olsun, g...|[{document, 0, 27...|[{document, 0, 27...|
|      Iyi olacağımı.|[{document, 0, 13...|[{document, 0, 13...|
|İdarede olan kişi...|[{d

In [14]:
result.printSchema()

root
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentences: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = tru

In [15]:
pd_res = result.select("sentences.result").toPandas()
pd_res

,result
0,[Neredesin?]
1,[Hapı yuttu!]
2,"[If Bu mekik, var]"
3,[- Bembeyaz olmuştu.]
4,[Onu bozguna uğratmıştık.]
...,...
960462,[Açığa çıkmak hoşuma gitmedi.]
960463,[Profiline dokunsana.]
960464,[Merhaba Erica.]
960465,[- Daha basit.]


In [ ]:
pd_res.to_csv("Spark_Sentence_Result1.csv", index=False)

In [ ]:
#pd_res = result.select("path","sentences.result").toPandas()
#pd_res

In [ ]:
#pd_res.loc[0].result